<a id='top'></a>

# Wyscout Parsing
##### Notebook to parse JSON Event data from [Wyscout](https://wyscout.com/).

### By [Edd Webster](https://www.twitter.com/eddwebster)
Last updated: 03/09/2020

![title](../../img/wyscout_logo.png)

Click [here](#section5) to jump straight to the Exploratory Data Analysis section and skip the [Task Brief](#section2), [Data Sources](#section3), and [Data Engineering](#section4) sections. Or click [here](#section6) to jump straight to the Conclusion.

___

<a id='sectionintro'></a>

## <a id='import_libraries'>Introduction</a>
This notebook parses pubicly available [Wyscout](https://wyscout.com/) football match data for the Big 5 European leagues for the 17/18 season, using [pandas](http://pandas.pydata.org/) for data manipulation through DataFrames.

For more information about this notebook and the author, I'm available through all the following channels:
*    [eddwebster.com](https://www.eddwebster.com/),
*    edd.j.webster@gmail.com,
*    [@eddwebster](https://www.twitter.com/eddwebster),
*    [LinkedIn.com/in/eddwebster](https://www.linkedin.com/in/eddwebster/),
*    [GitHub/eddwebster](https://github.com/eddwebster/),
*    [Kaggle.com/eddwebster](https://www.kaggle.com/eddwebster), and
*    [HackerRank.com/eddwebster](https://www.hackerrank.com/eddwebster).

The accompanying GitHub repository for this notebook can be found [here](https://github.com/eddwebster/fifa-league) and a static version of this notebook can be found [here](https://nbviewer.jupyter.org/github/eddwebster/fifa-league/blob/master/FIFA%2020%20Fantasy%20Football%20League%20using%20TransferMarkt%20Player%20Valuations.ipynb).

___

<a id='sectioncontents'></a>

## <a id='notebook_contents'>Notebook Contents</a>
1.    [Notebook Dependencies](#section1)<br>
2.    [Project Brief](#section2)<br>
3.    [Data Sources](#section3)<br>
      1.    [Introduction](#section3.1)<br>
      2.    [Competitions](#section3.2)<br>
      3.    [Events](#section3.3)<br>
      4.    [Matches](#section3.4)<br>
      5.    [Players](#section3.5)<br>
      6.    [Teams](#section3.6)<br>
4.    [Data Engineering](#section4)<br>
      1.    [Join Datasets](#section4.1)<br>
      5.    [Export DataFrame](#section4.5)<br>
5.    [Exploratory Data Analysis (EDA)](#section5)<br>
      1.    [...](#section5.1)<br>
      2.    [...](#section5.2)<br>
      3.    [...](#section5.3)<br>
6.    [Summary](#section6)<br>
7.    [Next Steps](#section7)<br>
8.    [Bibliography](#section8)<br>

___

<a id='section1'></a>

## <a id='#section1'>1. Notebook Dependencies</a>

This notebook was written using [Python 3](https://docs.python.org/3.7/) and requires the following libraries:
*    [`Jupyter notebooks`](https://jupyter.org/) for this notebook environment with which this project is presented;
*    [`NumPy`](http://www.numpy.org/) for multidimensional array computing;
*    [`pandas`](http://pandas.pydata.org/) for data analysis and manipulation; and
*    [`matplotlib`](https://matplotlib.org/contents.html?v=20200411155018) for data visualisations;

All packages used for this notebook except for BeautifulSoup can be obtained by downloading and installing the [Conda](https://anaconda.org/anaconda/conda) distribution, available on all platforms (Windows, Linux and Mac OSX). Step-by-step guides on how to install Anaconda can be found for Windows [here](https://medium.com/@GalarnykMichael/install-python-on-windows-anaconda-c63c7c3d1444) and Mac [here](https://medium.com/@GalarnykMichael/install-python-on-mac-anaconda-ccd9f2014072), as well as in the Anaconda documentation itself [here](https://docs.anaconda.com/anaconda/install/).

### Import Libraries and Modules

In [1]:
# Python ≥3.5 (ideally)
import platform
import sys, getopt
assert sys.version_info >= (3, 5)
import csv

# Import Dependencies
%matplotlib inline

# Math Operations
import numpy as np
from math import pi

# Datetime
import datetime
from datetime import date
import time

# Data Preprocessing
import pandas as pd    # version 1.0.3
import os    #  used to read the csv filenames
import re
import random
from io import BytesIO
from pathlib import Path

# Reading directories
import glob
import os

# Working with JSON
import json
from pandas.io.json import json_normalize
from ast import literal_eval

# Fuzzy Matching - Record Linkage
import recordlinkage
import jellyfish
import numexpr as ne

# Data Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-whitegrid')
import missingno as msno    # visually display missing data

# Display in Jupyter
from IPython.display import Image, YouTubeVideo
from IPython.core.display import HTML

# Ignore Warnings
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

print('Setup Complete')

Setup Complete


In [2]:
# Python / module versions used here for reference
print('Python: {}'.format(platform.python_version()))
print('NumPy: {}'.format(np.__version__))
print('pandas: {}'.format(pd.__version__))
print('matplotlib: {}'.format(mpl.__version__))
print('Seaborn: {}'.format(sns.__version__))

Python: 3.7.6
NumPy: 1.18.1
pandas: 1.0.1
matplotlib: 3.1.3
Seaborn: 0.10.0


### Defined Filepaths

In [3]:
# Set up initial paths to subfolders
base_dir = os.path.join('..', '..', )
data_dir = os.path.join(base_dir, 'data')
data_dir_fbref = os.path.join(base_dir, 'data', 'fbref')
data_dir_tm = os.path.join(base_dir, 'data', 'tm')
data_dir_wyscout = os.path.join(base_dir, 'data', 'wyscout')
img_dir = os.path.join(base_dir, 'img')
fig_dir = os.path.join(base_dir, 'img', 'fig')
video_dir = os.path.join(base_dir, 'video')

---

<a id='section2'></a>

## <a id='#section2'>2. Project Brief</a>
This Jupyter notebook explores how to parse pubicly available [Wyscout](https://wyscout.com/) JSON data of football matches for the Big 5 European leagues for the 17/18 season using [pandas](http://pandas.pydata.org/) for data manipulation through DataFrames.

The resulting five engineered DataFrames for each of the Big 5 European league are exported to CSV files. This data can be further analysed in Python, joined to other datasets, or explored using Tableau, PowerBI, Microsoft Excel.

---

<a id='section3'></a>

## <a id='#section3'>3. Data Sources</a>

### <a id='#section3.1'>3.1. Introduction</a>
[Wyscout](https://wyscout.com/) is an Italian company that supports football scouting, match analysis and transfer dynamics. The company was founded in Genoa, Italy in 2004 and provides video analysis tools and digital databases regarding performances and matches for coaches, teams and players dealing with football business.

The purpose is to allow them have a detailed sight of a large number of athletes about individual performances, patterns of play and tactical strategy.

This notebook explores a complete dataset of event data for the Big 5 European leagues during the 17/18 season.

A detailed description of the data can be found in the following paper:
- Pappalardo, L., Cintia, P., Rossi, A. et al. **A public data set of spatio-temporal match events in soccer competitions**. Scientific Data 6, 236 (2019) doi:10.1038/s41597-019-0247-7, https://www.nature.com/articles/s41597-019-0247-7

It is from this paper import the *matches*, *events*, *players*, *playerank*, *referees*, *coaches*, and *competition* data sets from the <a href="https://doi.org/10.6084/m9.figshare.c.4415000.v5">figshare repository</a>. Data are stored in `.json` format. 

The complete data set contains:
- 1,941 matches
- 3,251,294 events
- 4,299 players.

The datasets we will be using are:
- competitions;
- events;
- matches;
- players; and
- teams

Before conducting our EDA, the data needs to be imported as a DataFrame in the Data Sources section [Section 3](#section3) and cleaned in the Data Engineering section [Section 4](#section4).

We'll be using the [pandas](http://pandas.pydata.org/) library to import our data to this workbook as a DataFrame.

### <a id='#section3.2'>3.2. Competitions</a>
This data provides the details of the competitions in the dataset

#### <a id='#section3.2.1.'>3.2.1. Data Dictionary</a>
...

#### <a id='#section3.2.2.'>3.2.2. Import Data</a>

##### Import data

In [4]:
# Import the Wyscout JSON Competitions for the Big 5 European leagues for the 17/18 season
with open(data_dir_wyscout + '/raw/json/competitions.json') as f:
    json_wyscout_competitions_data = json.load(f)

In [5]:
# Display the Competitions JSON
json_wyscout_competitions_data

[{'name': 'Italian first division',
  'wyId': 524,
  'format': 'Domestic league',
  'area': {'name': 'Italy',
   'id': '380',
   'alpha3code': 'ITA',
   'alpha2code': 'IT'},
  'type': 'club'},
 {'name': 'English first division',
  'wyId': 364,
  'format': 'Domestic league',
  'area': {'name': 'England',
   'id': '0',
   'alpha3code': 'XEN',
   'alpha2code': ''},
  'type': 'club'},
 {'name': 'Spanish first division',
  'wyId': 795,
  'format': 'Domestic league',
  'area': {'name': 'Spain',
   'id': '724',
   'alpha3code': 'ESP',
   'alpha2code': 'ES'},
  'type': 'club'},
 {'name': 'French first division',
  'wyId': 412,
  'format': 'Domestic league',
  'area': {'name': 'France',
   'id': '250',
   'alpha3code': 'FRA',
   'alpha2code': 'FR'},
  'type': 'club'},
 {'name': 'German first division',
  'wyId': 426,
  'format': 'Domestic league',
  'area': {'name': 'Germany',
   'id': '276',
   'alpha3code': 'DEU',
   'alpha2code': 'DE'},
  'type': 'club'},
 {'name': 'European Championship',
 

##### Flatten Competitions data

In [6]:
# Flatten the Wyscout JSON Competition data and export the DataFrame as a CSV file
df_wyscout_competitions_data_flat = json_normalize(json_wyscout_competitions_data)
df_wyscout_competitions_data_flat.to_csv(data_dir_wyscout + '/raw/csv/competitions.csv', index=None, header=True)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


In [7]:
df_wyscout_competitions_data_flat

,name,wyId,format,type,area.name,area.id,area.alpha3code,area.alpha2code
0,Italian first division,524,Domestic league,club,Italy,380,ITA,IT
1,English first division,364,Domestic league,club,England,0,XEN,
2,Spanish first division,795,Domestic league,club,Spain,724,ESP,ES
3,French first division,412,Domestic league,club,France,250,FRA,FR
4,German first division,426,Domestic league,club,Germany,276,DEU,DE
5,European Championship,102,International cup,international,,0,XEU,
6,World Cup,28,International cup,international,,0,XWO,


##### Rename columns

In [8]:
df_wyscout_competitions_data_flat = df_wyscout_competitions_data_flat.rename(columns={'name': 'competition_name_generic',
                                                                                      'wyId': 'wyId',
                                                                                      'format': 'competition_format',
                                                                                      'type': 'competition_type',
                                                                                      'area.name': 'country',
                                                                                      'area.id': 'country_id',
                                                                                      'area.alpha3code': 'country_alpha3code',
                                                                                      'area.alpha2code': 'country_alpha2code'
                                                                                     }
                                                                            )

##### Create `league_name` column derived from the `competition_name_generic` column

In [9]:
dict_league_name_wyscout = {
    'Italy first division': 'Seria A',
    'English first division': 'Premier League',
    'Spanish first division': 'La Liga',
    'French first division': 'Ligue 1',
    'German first division': 'Bundesliga',
    'European Championship': 'UEFA EURO',
    'World Cup': 'FIFA World Cup'
}

In [10]:
df_wyscout_competitions_data_flat['league_name'] = df_wyscout_competitions_data_flat['competition_name_generic'].map(dict_league_name_wyscout)

##### Streamline the DataFrame

In [11]:
df_wyscout_competitions_data_flat.columns

Index(['competition_name_generic', 'wyId', 'competition_format',
       'competition_type', 'country', 'country_id', 'country_alpha3code',
       'country_alpha2code', 'league_name'],
      dtype='object')

In [12]:
# Select columns of interest
cols_competitions = ['league_name', 'wyId', 'country']

# Create more concise DataFrame using only columns of interest
df_wyscout_competitions_data_flat_select = df_wyscout_competitions_data_flat[cols_competitions]

# Export DataFrame
df_wyscout_competitions_data_flat_select.to_csv(data_dir_wyscout + '/raw/csv/competitions_select.csv', index=None, header=True)

In [13]:
df_wyscout_competitions_data_flat_select

,league_name,wyId,country
0,NaN,524,Italy
1,Premier League,364,England
2,La Liga,795,Spain
3,Ligue 1,412,France
4,Bundesliga,426,Germany
5,UEFA EURO,102,
6,FIFA World Cup,28,


For our analysis, we want just the Big 5 European leagues which are the following competition IDs:
*    Seria A - 380
*    Premier League - 364
*    La Liga - 724
*    Ligue 1 - 250
*    Bundesliga - 276

### <a id='#section3.3.'>3.3. Events</a>

#### <a id='#section3.3.1.'>3.3.1. Data Dictionary</a>
The [WyScout](https://www.transfermarkt.co.uk/) Events dataset has twelve features (columns) with the following definitions and data types:

| Feature     | Data type    |
|------|-----|
| `eventId`    | int64     |
| `subEventName`    | object     |
| `tags`    | object     |
| `playerId`    | int64     |
| `positions`    | object     |
| `matchId`    | int64     |
| `eventName`    | object     |
| `teamId`    | int64     |
| `matchPeriod`    | object     |
| `eventSec`    | float64     |
| `subEventId`    | object     |
| `id`    | int64     |

Refer to the Wyscout [API docs](https://apidocs.wyscout.com/) and [Events Manual](https://footballdata.wyscout.com/wp-content/uploads/2018/03/Wyscout-Events-Manual.pdf) for further information about event and subevents.

#### <a id='#section3.3.2.'>3.3.2. Import Data</a>

##### Read in Event Data

In [14]:
# Import all Wyscout JSON Events data for the Big 5 European leagues for the 17/18 season

## Premier League - England
with open(data_dir_wyscout + '/raw/json/events/events_England.json') as f:
    json_wyscout_event_data_england = json.load(f)
    
## Ligue 1 - France
with open(data_dir_wyscout + '/raw/json/events/events_France.json') as f:
    json_wyscout_event_data_france  = json.load(f)

## Bundesliga - Germany
with open(data_dir_wyscout + '/raw/json/events/events_Germany.json') as f:
    json_wyscout_event_data_germany = json.load(f)

## Seria A - Italy
with open(data_dir_wyscout + '/raw/json/events/events_Italy.json') as f:
    json_wyscout_event_data_italy = json.load(f)

## La Liga - Spain
with open(data_dir_wyscout + '/raw/json/events/events_Spain.json') as f:
    json_wyscout_event_data_spain = json.load(f)

In [15]:
json_wyscout_event_data_england 

[{'eventId': 8,
  'subEventName': 'Simple pass',
  'tags': [{'id': 1801}],
  'playerId': 25413,
  'positions': [{'y': 49, 'x': 49}, {'y': 78, 'x': 31}],
  'matchId': 2499719,
  'eventName': 'Pass',
  'teamId': 1609,
  'matchPeriod': '1H',
  'eventSec': 2.7586489999999912,
  'subEventId': 85,
  'id': 177959171},
 {'eventId': 8,
  'subEventName': 'High pass',
  'tags': [{'id': 1801}],
  'playerId': 370224,
  'positions': [{'y': 78, 'x': 31}, {'y': 75, 'x': 51}],
  'matchId': 2499719,
  'eventName': 'Pass',
  'teamId': 1609,
  'matchPeriod': '1H',
  'eventSec': 4.946850000000012,
  'subEventId': 83,
  'id': 177959172},
 {'eventId': 8,
  'subEventName': 'Head pass',
  'tags': [{'id': 1801}],
  'playerId': 3319,
  'positions': [{'y': 75, 'x': 51}, {'y': 71, 'x': 35}],
  'matchId': 2499719,
  'eventName': 'Pass',
  'teamId': 1609,
  'matchPeriod': '1H',
  'eventSec': 6.54218800000001,
  'subEventId': 82,
  'id': 177959173},
 {'eventId': 8,
  'subEventName': 'Head pass',
  'tags': [{'id': 180

##### Flatten Events data

In [16]:
# Flatten the JSON Events data for each of the Big 5 European leagues and export each DataFrame as a CSV file

## Premier League - England
df_wyscout_event_data_england_flat = json_normalize(json_wyscout_event_data_england)
df_wyscout_event_data_england_flat.to_csv(data_dir_wyscout + '/raw/csv/events/events_England.csv', index=None, header=True)
    
## Ligue 1 - France
df_wyscout_event_data_france_flat = json_normalize(json_wyscout_event_data_france)
df_wyscout_event_data_france_flat.to_csv(data_dir_wyscout + '/raw/csv/events/events_France.csv', index=None, header=True)

## Bundesliga - Germany
df_wyscout_event_data_germany_flat = json_normalize(json_wyscout_event_data_germany)
df_wyscout_event_data_germany_flat.to_csv(data_dir_wyscout + '/raw/csv/events/events_Germany.csv', index=None, header=True)

## Seria A - Italy
df_wyscout_event_data_italy_flat = json_normalize(json_wyscout_event_data_italy)
df_wyscout_event_data_italy_flat.to_csv(data_dir_wyscout + '/raw/csv/events/events_Italy.csv', index=None, header=True)

## La Liga - Spain
df_wyscout_event_data_spain_flat = json_normalize(json_wyscout_event_data_spain)
df_wyscout_event_data_spain_flat.to_csv(data_dir_wyscout + '/raw/csv/events/events_Spain.csv', index=None, header=True)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if sys.path[0] == '':
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


In [17]:
df_wyscout_event_data_england_flat.shape

(643150, 12)

##### Concatenate the flattened Events data for the Big 5 European leagues

In [18]:
# Concatenate the flattened events data for the Big 5 European leagues

## List of the Big 5 DataFrames
lst_events_dataframes_big5 = [df_wyscout_event_data_england_flat, df_wyscout_event_data_france_flat, df_wyscout_event_data_germany_flat, df_wyscout_event_data_italy_flat, df_wyscout_event_data_spain_flat]

## Concatenate the individual Big 5 DataFrames to one unified DataFrame
df_wyscout_event_data_big5_flat = pd.concat(lst_events_dataframes_big5)

## Export unified DataFrame
df_wyscout_event_data_big5_flat.to_csv(data_dir_wyscout + '/raw/csv/events/events_big5.csv', index=None, header=True)

In [19]:
df_wyscout_event_data_big5_flat.shape

(3071395, 12)

### <a id='#section3.4.'>3.4. Matches</a>

##### Read in Matches Data

In [20]:
# Import all Wyscout JSON matches data from the Big 5 European leagues for the 17/18 season

## Premier League - England
with open(data_dir_wyscout + '/raw/json/matches/matches_England.json') as f:
    json_wyscout_matches_data_england = json.load(f)
    
## Ligue 1 - France
with open(data_dir_wyscout + '/raw/json/matches/matches_France.json') as f:
    json_wyscout_matches_data_france  = json.load(f)

## Bundesliga - Germany
with open(data_dir_wyscout + '/raw/json/matches/matches_Germany.json') as f:
    json_wyscout_matches_data_germany = json.load(f)

## Seria A - Italy
with open(data_dir_wyscout + '/raw/json/matches/matches_Italy.json') as f:
    json_wyscout_matches_data_italy = json.load(f)

## La Liga - Spain
with open(data_dir_wyscout + '/raw/json/matches/matches_Spain.json') as f:
    json_wyscout_matches_data_spain = json.load(f)

##### Flatten Matches data

In [21]:
# Flatten the JSON Matches data for each of the Big 5 European leagues and export each DataFrame as a CSV file

## Premier League - England
df_wyscout_matches_data_england_flat = json_normalize(json_wyscout_matches_data_england)
df_wyscout_matches_data_england_flat.to_csv(data_dir_wyscout + '/raw/csv/matches/matches_England.csv', index=None, header=True)
    
## Ligue 1 - France
df_wyscout_matches_data_france_flat = json_normalize(json_wyscout_matches_data_france)
df_wyscout_matches_data_france_flat.to_csv(data_dir_wyscout + '/raw/csv/matches/matches_France.csv', index=None, header=True)

## Bundesliga - Germany
df_wyscout_matches_data_germany_flat = json_normalize(json_wyscout_matches_data_germany)
df_wyscout_matches_data_germany_flat.to_csv(data_dir_wyscout + '/raw/csv/matches/matches_Germany.csv', index=None, header=True)

## Seria A - Italy
df_wyscout_matches_data_italy_flat = json_normalize(json_wyscout_matches_data_italy)
df_wyscout_matches_data_italy_flat.to_csv(data_dir_wyscout + '/raw/csv/matches/matches_Italy.csv', index=None, header=True)

## La Liga - Spain
df_wyscout_matches_data_spain_flat = json_normalize(json_wyscout_matches_data_spain)
df_wyscout_matches_data_spain_flat.to_csv(data_dir_wyscout + '/raw/csv/matches/matches_Spain.csv', index=None, header=True)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if sys.path[0] == '':
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


##### Concatenate the flattened Matches data for the Big 5 European leagues

In [22]:
# Concatenate the flattened events data for the Big 5 European leagues

## List of the Big 5 DataFrames
lst_matches_dataframes_big5 = [df_wyscout_matches_data_england_flat, df_wyscout_matches_data_france_flat, df_wyscout_matches_data_germany_flat, df_wyscout_matches_data_italy_flat, df_wyscout_matches_data_spain_flat]

## Concatenate the individual Big 5 DataFrames to one unified DataFrame
df_wyscout_matches_data_big5_flat = pd.concat(lst_matches_dataframes_big5)

## Export unified DataFrame
df_wyscout_matches_data_big5_flat.to_csv(data_dir_wyscout + '/raw/csv/matches/matches_big5.csv', index=None, header=True)

In [23]:
df_wyscout_matches_data_big5_flat

,status,roundId,gameweek,seasonId,dateutc,winner,venue,wyId,label,date,...,teamsData.695.coachId,teamsData.695.side,teamsData.695.teamId,teamsData.695.score,teamsData.695.scoreP,teamsData.695.hasFormation,teamsData.695.formation.bench,teamsData.695.formation.lineup,teamsData.695.formation.substitutions,teamsData.695.scoreHT
0,Played,4405654,38,181150,2018-05-13 14:00:00,1659,Turf Moor,2500089,"Burnley - AFC Bournemouth, 1 - 2","May 13, 2018 at 4:00:00 PM GMT+2",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Played,4405654,38,181150,2018-05-13 14:00:00,1628,Selhurst Park,2500090,"Crystal Palace - West Bromwich Albion, 2 - 0","May 13, 2018 at 4:00:00 PM GMT+2",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Played,4405654,38,181150,2018-05-13 14:00:00,1609,The John Smith's Stadium,2500091,"Huddersfield Town - Arsenal, 0 - 1","May 13, 2018 at 4:00:00 PM GMT+2",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Played,4405654,38,181150,2018-05-13 14:00:00,1612,Anfield,2500092,"Liverpool - Brighton & Hove Albion, 4 - 0","May 13, 2018 at 4:00:00 PM GMT+2",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Played,4405654,38,181150,2018-05-13 14:00:00,1611,Old Trafford,2500093,"Manchester United - Watford, 1 - 0","May 13, 2018 at 4:00:00 PM GMT+2",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,Played,4406122,1,181144,2017-08-19 20:15:00,0,Estadio Ram\u00f3n S\u00e1nchez Pizju\u00e1n,2565552,"Sevilla - Espanyol, 1 - 1","August 19, 2017 at 10:15:00 PM GMT+2",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
376,Played,4406122,1,181144,2017-08-19 18:15:00,0,Estadi Municipal de Montilivi,2565551,"Girona - Atl\u00e9tico Madrid, 2 - 2","August 19, 2017 at 8:15:00 PM GMT+2",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
377,Played,4406122,1,181144,2017-08-19 16:15:00,687,Estadio de Bala\u00eddos,2565549,"Celta de Vigo - Real Sociedad, 2 - 3","August 19, 2017 at 6:15:00 PM GMT+2",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
378,Played,4406122,1,181144,2017-08-18 20:15:00,674,Estadio de Mestalla,2565556,"Valencia - Las Palmas, 1 - 0","August 18, 2017 at 10:15:00 PM GMT+2",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
# Displays all columns
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_wyscout_matches_data_big5_flat.dtypes)

status                                      object
roundId                                      int64
gameweek                                     int64
seasonId                                     int64
dateutc                                     object
winner                                       int64
venue                                       object
wyId                                         int64
label                                       object
date                                        object
referees                                    object
duration                                    object
competitionId                                int64
teamsData.1646.scoreET                     float64
teamsData.1646.coachId                     float64
teamsData.1646.side                         object
teamsData.1646.teamId                      float64
teamsData.1646.score                       float64
teamsData.1646.scoreP                      float64
teamsData.1646.hasFormation    

##### Streamline the DataFrame
The `Matches` data includes a lot of `teamsData` columns that we don't require at this stage. We will streamline this DataFrame to only include the columns of interest.

In [25]:
# Select columns of interest
cols_matches = ['status', 'roundId', 'gameweek', 'seasonId', 'dateutc', 'winner', 'venue', 'wyId', 'label', 'date', 'referees', 'duration', 'competitionId']

# Create more concise DataFrame using only columns of interest
df_wyscout_matches_data_big5_flat_select = df_wyscout_matches_data_big5_flat[cols_matches]

### <a id='#section3.5.'>3.5. Players</a>

##### Read in the Players data

In [26]:
# Import the Wyscout JSON Players data from the Big 5 European leagues for the 17/18 season
with open(data_dir_wyscout + '/raw/json/players.json') as f:
    json_wyscout_players_data = json.load(f)

In [27]:
json_wyscout_players_data

[{'passportArea': {'name': 'Turkey',
   'id': '792',
   'alpha3code': 'TUR',
   'alpha2code': 'TR'},
  'weight': 78,
  'firstName': 'Harun',
  'middleName': '',
  'lastName': 'Tekin',
  'currentTeamId': 4502,
  'birthDate': '1989-06-17',
  'height': 187,
  'role': {'code2': 'GK', 'code3': 'GKP', 'name': 'Goalkeeper'},
  'birthArea': {'name': 'Turkey',
   'id': '792',
   'alpha3code': 'TUR',
   'alpha2code': 'TR'},
  'wyId': 32777,
  'foot': 'right',
  'shortName': 'H. Tekin',
  'currentNationalTeamId': 4687},
 {'passportArea': {'name': 'Senegal',
   'id': '686',
   'alpha3code': 'SEN',
   'alpha2code': 'SN'},
  'weight': 73,
  'firstName': 'Malang',
  'middleName': '',
  'lastName': 'Sarr',
  'currentTeamId': 3775,
  'birthDate': '1999-01-23',
  'height': 182,
  'role': {'code2': 'DF', 'code3': 'DEF', 'name': 'Defender'},
  'birthArea': {'name': 'France',
   'id': '250',
   'alpha3code': 'FRA',
   'alpha2code': 'FR'},
  'wyId': 393228,
  'foot': 'left',
  'shortName': 'M. Sarr',
  'cur

##### Flatten Players data

In [28]:
# Flatten the JSON Players for the Big 5 European leagues and export DataFrame as a CSV file
df_wyscout_players_data_flat = json_normalize(json_wyscout_players_data)
df_wyscout_players_data_flat.to_csv(data_dir_wyscout + '/raw/csv/players.csv', index=None, header=True)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


In [29]:
df_wyscout_players_data_flat.head()

,weight,firstName,middleName,lastName,currentTeamId,birthDate,height,wyId,foot,shortName,...,passportArea.id,passportArea.alpha3code,passportArea.alpha2code,role.code2,role.code3,role.name,birthArea.name,birthArea.id,birthArea.alpha3code,birthArea.alpha2code
0,78,Harun,,Tekin,4502,1989-06-17,187,32777,right,H. Tekin,...,792,TUR,TR,GK,GKP,Goalkeeper,Turkey,792,TUR,TR
1,73,Malang,,Sarr,3775,1999-01-23,182,393228,left,M. Sarr,...,686,SEN,SN,DF,DEF,Defender,France,250,FRA,FR
2,72,Over,,Mandanda,3772,1998-10-26,176,393230,,O. Mandanda,...,250,FRA,FR,GK,GKP,Goalkeeper,France,250,FRA,FR
3,82,Alfred John Momar,,N'Diaye,683,1990-03-06,187,32793,right,A. N'Diaye,...,686,SEN,SN,MD,MID,Midfielder,France,250,FRA,FR
4,84,Ibrahima,,Konat\u00e9,2975,1999-05-25,192,393247,right,I. Konat\u00e9,...,250,FRA,FR,DF,DEF,Defender,France,250,FRA,FR


In [30]:
# Displays all columns
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_wyscout_players_data_flat.dtypes)

weight                      int64
firstName                  object
middleName                 object
lastName                   object
currentTeamId              object
birthDate                  object
height                      int64
wyId                        int64
foot                       object
shortName                  object
currentNationalTeamId      object
passportArea.name          object
passportArea.id            object
passportArea.alpha3code    object
passportArea.alpha2code    object
role.code2                 object
role.code3                 object
role.name                  object
birthArea.name             object
birthArea.id               object
birthArea.alpha3code       object
birthArea.alpha2code       object
dtype: object


##### Streamline the DataFrame
Include only the columns of interest

In [31]:
# Select columns of interest
cols_players = ['weight', 'firstName', 'middleName', 'lastName', 'birthDate', 'height', 'wyId', 'foot', 'shortName', 'passportArea.name', 'role.name', 'birthArea.name']

# Create more concise DataFrame using only columns of interest
df_wyscout_players_data_flat_select = df_wyscout_players_data_flat[cols_players]

##### Create `fullName` attribute 
Join together the `firstName` and `lastName` attribute to create a `fullName` attribute.

In [32]:
df_wyscout_players_data_flat_select['fullName'] = df_wyscout_players_data_flat_select['firstName'].astype(str) + ' ' + df_wyscout_players_data_flat_select['lastName'].astype(str)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [33]:
df_wyscout_players_data_flat_select

,weight,firstName,middleName,lastName,birthDate,height,wyId,foot,shortName,passportArea.name,role.name,birthArea.name,fullName
0,78,Harun,,Tekin,1989-06-17,187,32777,right,H. Tekin,Turkey,Goalkeeper,Turkey,Harun Tekin
1,73,Malang,,Sarr,1999-01-23,182,393228,left,M. Sarr,Senegal,Defender,France,Malang Sarr
2,72,Over,,Mandanda,1998-10-26,176,393230,,O. Mandanda,France,Goalkeeper,France,Over Mandanda
3,82,Alfred John Momar,,N'Diaye,1990-03-06,187,32793,right,A. N'Diaye,Senegal,Midfielder,France,Alfred John Momar N'Diaye
4,84,Ibrahima,,Konat\u00e9,1999-05-25,192,393247,right,I. Konat\u00e9,France,Defender,France,Ibrahima Konat\u00e9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3598,72,Ali,,Ma\u00e2loul,1990-01-01,175,120839,left,A. Ma\u00e2loul,Tunisia,Defender,Tunisia,Ali Ma\u00e2loul
3599,76,Carlos Alberto,,C\u00e1ceda Oyaguez,1991-09-27,183,114736,right,C. C\u00e1ceda,Peru,Goalkeeper,Peru,Carlos Alberto C\u00e1ceda Oyaguez
3600,78,Miguel Gianpierre,,Araujo Blanco,1994-10-24,179,114908,right,M. Araujo,Peru,Defender,Peru,Miguel Gianpierre Araujo Blanco
3601,70,Ahmed Reda,,Tagnaouti,1996-04-05,182,285583,right,A. Tagnaouti,Morocco,Goalkeeper,Morocco,Ahmed Reda Tagnaouti


##### Export DataFrame

In [34]:
df_wyscout_players_data_flat_select.to_csv(data_dir_wyscout + '/raw/csv/players_select.csv', index=None, header=True)

##### Investigate problem with player names

In [35]:
lst_players = list(df_wyscout_players_data_flat_select['fullName'].unique())

Player names with characters need cleaning e.g. 'Giovani dos Santos Ram\\u00edrez' is 'Giovani dos Santos Ramírez'.

### <a id='#section3.6'>3.6. Teams</a>

##### Read in the Teams data

In [36]:
# Import the Wyscout JSON Teams data from the Big 5 European leagues for the 17/18 season
with open(data_dir_wyscout + '/raw/json/teams.json') as f:
    json_wyscout_teams_data = json.load(f)

In [37]:
json_wyscout_teams_data

[{'city': 'Newcastle upon Tyne',
  'name': 'Newcastle United',
  'wyId': 1613,
  'officialName': 'Newcastle United FC',
  'area': {'name': 'England',
   'id': '0',
   'alpha3code': 'XEN',
   'alpha2code': ''},
  'type': 'club'},
 {'city': 'Vigo',
  'name': 'Celta de Vigo',
  'wyId': 692,
  'officialName': 'Real Club Celta de Vigo',
  'area': {'name': 'Spain',
   'id': '724',
   'alpha3code': 'ESP',
   'alpha2code': 'ES'},
  'type': 'club'},
 {'city': 'Barcelona',
  'name': 'Espanyol',
  'wyId': 691,
  'officialName': 'Reial Club Deportiu Espanyol',
  'area': {'name': 'Spain',
   'id': '724',
   'alpha3code': 'ESP',
   'alpha2code': 'ES'},
  'type': 'club'},
 {'city': 'Vitoria-Gasteiz',
  'name': 'Deportivo Alav\\u00e9s',
  'wyId': 696,
  'officialName': 'Deportivo Alav\\u00e9s',
  'area': {'name': 'Spain',
   'id': '724',
   'alpha3code': 'ESP',
   'alpha2code': 'ES'},
  'type': 'club'},
 {'city': 'Valencia',
  'name': 'Levante',
  'wyId': 695,
  'officialName': 'Levante UD',
  'area':

##### Flatten Teams data

In [38]:
# Flatten the JSON Teams data for the Big 5 European leagues and export DataFrame as a CSV file
df_wyscout_teams_data_flat = json_normalize(json_wyscout_teams_data)
df_wyscout_teams_data_flat.to_csv(data_dir_wyscout + '/raw/csv/players.csv', index=None, header=True)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


In [39]:
df_wyscout_teams_data_flat.head()

,city,name,wyId,officialName,type,area.name,area.id,area.alpha3code,area.alpha2code
0,Newcastle upon Tyne,Newcastle United,1613,Newcastle United FC,club,England,0,XEN,
1,Vigo,Celta de Vigo,692,Real Club Celta de Vigo,club,Spain,724,ESP,ES
2,Barcelona,Espanyol,691,Reial Club Deportiu Espanyol,club,Spain,724,ESP,ES
3,Vitoria-Gasteiz,Deportivo Alav\u00e9s,696,Deportivo Alav\u00e9s,club,Spain,724,ESP,ES
4,Valencia,Levante,695,Levante UD,club,Spain,724,ESP,ES


##### Streamline the DataFrame
Include only the columns of interest

In [40]:
df_wyscout_teams_data_flat.columns

Index(['city', 'name', 'wyId', 'officialName', 'type', 'area.name', 'area.id',
       'area.alpha3code', 'area.alpha2code'],
      dtype='object')

##### Rename columns

In [41]:
# Select columns of interest
cols_teams = ['name', 'wyId']

# Create more concise DataFrame using only columns of interest
df_wyscout_teams_data_flat_select = df_wyscout_teams_data_flat[cols_teams]

In [42]:
df_wyscout_teams_data_flat_select = df_wyscout_teams_data_flat_select.rename(columns={'name': 'teamName'})

##### Export DataFrame

In [43]:
df_wyscout_teams_data_flat_select.to_csv(data_dir_wyscout + '/raw/csv/teams_select.csv', index=None, header=True)

---

## <a id='#section4'>4. Data Engineering</a>

### <a id='#section4.1'>4.1. Join Datasets</a>
Next, we're required to join the `Matches` DataFrame and the `Players` DataFrame to the `Events` DatFrame. The `Events` data is the base DataFrame in which we join the other tables via `wyId`, `matchId`, `competitionId`, `playerId`, and `teamId`.

##### Join Matches DataFrame to Events DataFrame

In [44]:
# Join the Events DataFrame to the Matches DataFrame
df_wyscout_merge = pd.merge(df_wyscout_event_data_big5_flat, df_wyscout_matches_data_big5_flat_select, left_on='matchId', right_on='wyId')

##### Join Competitions DataFrame to Merged DataFrame via Events DataFrame

In [45]:
# Join the Players DataFrame to the Events-Matches DataFrame
df_wyscout_merge = pd.merge(df_wyscout_merge, df_wyscout_competitions_data_flat_select, left_on='competitionId', right_on='wyId')

##### Join Player DataFrame to Merged DataFrame via Events DataFrame

In [46]:
# Join the Players DataFrame to the Events-Matches DataFrame
df_wyscout_merge = pd.merge(df_wyscout_merge, df_wyscout_players_data_flat_select, left_on='playerId', right_on='wyId')

##### Join Teams DataFrame to Merged DataFrame via Events DataFrame

In [47]:
# Join the Players DataFrame to the Events-Matches DataFrame
df_wyscout_merge = pd.merge(df_wyscout_merge, df_wyscout_teams_data_flat_select, left_on='teamId', right_on='wyId')

In [48]:
df_wyscout_merge.shape

(2845357, 43)

### <a id='#section4.2'>4.2. Export DataFrame</a>

In [49]:
df_wyscout_merge.to_csv(data_dir_wyscout + '/raw/csv/combined/wyscout_big5_combined.csv', index=None, header=True)

---

## <a id='#section5'>5. Exploratory Data Analysis</a>
...

---

## <a id='#section6'>6. Summary</a>
This notebook parses Wyscout data using [pandas](http://pandas.pydata.org/) for data maniuplation through DataFrames.

---

## <a id='#section7'>7. Next Steps</a>
...

---

## <a id='#section8'>8. References</a>

### <a id='#section9.1'>8.1. Bibliography</a>

#### Wyscout
*    WyScout: https://wyscout.com/
*    WyScout Events data manual: https://footballdata.wyscout.com/events-manual/
*    Pappalardo, Luca; Massucco, Emanuele (2019): Soccer match event dataset. figshare. Collection. https://doi.org/10.6084/m9.figshare.c.4415000.v5
*    Pappalardo, L., Cintia, P., Rossi, A. et al. **A public data set of spatio-temporal match events in soccer competitions**. Scientific Data 6, 236 (2019) doi:10.1038/s41597-019-0247-7, https://www.nature.com/articles/s41597-019-0247-7

### <a id='#section9.2'>8.2. Python Techniques Observed</a>
To conduct our analysis, we have used the following libraries and modules for the following tasks:
*    [NumPy](http://www.numpy.org/) for multidimensional array computing,
*    [pandas](http://pandas.pydata.org/) for data manipulation and ingestion

We have also demonstrated an array of techniques in Python using the following methods and functions:
*    pandas EDA methods:
     +    [head()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.head.html),
     +    [tail()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.tail.html),
     +    [shape](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dtypes.html),
     +    [columns](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.columns.html),
     +    [dtypes](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dtypes.html),
     +    [info](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.info.html), and
     +    [describe](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html).
*    The [missingno](https://pypi.org/project/missingno/) library to visualise how many missing values we have in the dataset, and
*    The pandas [.to_csv()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html) method to export the DataFrames as csv files.

---

***Visit my website [EddWebster.com](https://www.eddwebster.com) or my [GitHub Repository](https://github.com/eddwebster) for more projects. If you'd like to get in contact, my Twitter handle is [@eddwebster](http://www.twitter.com/eddwebster) and my email is: edd.j.webster@gmail.com.***

[Back to the top](#top)